In [ ]:
pip install scikit-learn pandas numpy requests geopy fuzzywuzzy


In [ ]:
import joblib
import pandas as pd
from geolocation_model import GeolocationModel
from nearby_services_model import NearbyServicesModel

def load_model_and_vectorizer(model_path, vectorizer_path):
    """
    Load the model and vectorizer from joblib files.
    """
    model = joblib.load(model_path)
    vectorizer = joblib.load(vectorizer_path)
    return model, vectorizer

def load_remedies_and_otc():
    """
    Load home remedies and OTC medicines from CSV files.
    """
    remedies_df = pd.read_csv('REMEDIES.csv')  # Ensure this file exists
    otc_df = pd.read_csv('Book1__OTC.csv')      # Ensure this file exists
    return remedies_df, otc_df

def get_remedies_for_disease(disease, remedies_df):
    """
    Get home remedies for the specified disease from the DataFrame.
    """
    # Normalize the disease name for comparison
    remedies_row = remedies_df[remedies_df['DISEASE NAME'].str.lower().str.strip() == disease.lower().strip()]
    if not remedies_row.empty:
        # Extract remedies from the row
        remedies = remedies_row.iloc[0, 1:7].dropna().tolist()  # Get columns HOMEREMEDY1 to HOMEREMEDY6
        return remedies
    return []

def get_otc_for_disease(disease, otc_df):
    """
    Get OTC medicines for the specified disease from the DataFrame.
    """
    # Normalize the disease name for comparison
    otc_row = otc_df[otc_df['Diseases'].str.lower().str.strip() == disease.lower().strip()]
    if not otc_row.empty:
        # Extract OTC medicines from the row
        otc_medicines = otc_row.iloc[0, 1:5].dropna().tolist()  # Get columns OTC1 to OTC4
        return otc_medicines
    return []

def process_user_input():
    """
    Main function to process user input and provide health-related suggestions.
    """
    # Load models and vectorizers
    symptom_model, symptom_vectorizer = load_model_and_vectorizer('naive_bayes_modelS.pkl', 'tfidf_vectorizerS.pkl')
    geolocation_model = GeolocationModel()
    services_model = NearbyServicesModel(api_key="vTUXwsGD6SPVv_cpCDTdI_TPs4IPwdxSbyB_yMjy3W4")

    # Load remedies and OTC medicines from CSV
    remedies_df, otc_df = load_remedies_and_otc()

    print("Hello! I am your health assistant.")

    user_input = input("Please tell me your symptoms (e.g., headache, fever, etc.): ")

    # Transform user input using TF-IDF vectorizer and then use Naive Bayes model to predict symptoms
    user_input_vectorized = symptom_vectorizer.transform([user_input])
    matched_symptoms = symptom_model.predict(user_input_vectorized)

    if matched_symptoms:
        detected_symptoms = [symptom for symptom in matched_symptoms if isinstance(symptom, str)]
        print(f"Detected symptoms: {', '.join(set(detected_symptoms))}")

        # Use the detected symptoms as the disease name
        disease = detected_symptoms[0]  # Assuming the first matched symptom is the disease
        print(f"I think you might have {disease}.")  # Debugging line

        # Get home remedies for the detected disease
        remedies = get_remedies_for_disease(disease, remedies_df)
        print("Predicted remedies:", remedies)  # Debugging line

        if remedies:
            print("Here are some home remedies:")
            for remedy in remedies:
                print(f"- {remedy}")
        else:
            print("No home remedies found for this disease.")

        # Initialize otc_medicines variable
        otc_medicines = []  # Initialize to an empty list

        # Get OTC medicines for the detected disease

        otc_choice = input("Do you want to know about OTC medicines for this? (yes/no): ").lower()
        if otc_choice == 'yes':
            otc_medicines = get_otc_for_disease(disease, otc_df)
        if otc_medicines:
            print("Here are some OTC medicines you can try:")
            for otc in otc_medicines:
                print(f"- {otc}")
            else:
                print("No OTC medicines found.")
        elif otc_choice == 'no':
                print("No OTC medicines will be displayed.")
        else:
                print("Invalid input. Please enter 'yes' or 'no'.")

        location = input("Please provide your address or city for nearby medical services: ")
        lat, lng = geolocation_model.get_geolocation(location)
        if lat and lng:
              services_model.get_nearby_services(lat, lng)
        else:
              print("Sorry, I couldn't find that location. Please try again.")

process_user_input()


Hello! I am your health assistant.
Please tell me your symptoms (e.g., headache, fever, etc.): I have a headache
Detected symptoms: Eye Strain
I think you might have Eye Strain.
Predicted remedies: ['20-20-20 Rule\nWhy to Use:\nReduces strain by giving your eyes regular breaks.\n\nHow to Use:\n\nEvery 20 minutes, look at something 20 feet away for 20 seconds.', 'Cold Compress\nWhy to Use:\nReduces eye fatigue and soothes tired eyes.\n\nHow to Use:\n\nPlace a cold washcloth or an ice pack wrapped in cloth over your eyes for 10 minutes.', 'Warm Compress\nWhy to Use:\nRelaxes eye muscles and improves blood circulation.\n\nHow to Use:\n\nSoak a clean cloth in warm water, wring it out, and place it over your closed eyes for 10 minutes.', 'Palming Exercise\nWhy to Use:\nCalms eye muscles and relieves tension.\n\nHow to Use:\n\nRub your palms together until warm, then cup them over closed eyes without applying pressure.', 'Blinking Exercise\nWhy to Use:\nMoistens eyes and prevents dryness.\n\